# VELMA simulated runoff

In [87]:
%matplotlib widget

import __init__
import scripts.config as config
import numpy as np
import pandas as pd
import tempfile
import datetime
import ipywidgets as widgets
from ipywidgets import interact
from natsort import natsorted
import os
from sklearn.metrics import mean_squared_error, r2_score
from matplotlib.font_manager import FontProperties
import seaborn as sns
# import matplotlib as mpl
import matplotlib.pyplot as plt
import importlib

In [88]:
# Plotting parameters

XSMALL_SIZE = 6
SMALL_SIZE = 7
MEDIUM_SIZE = 9
BIGGER_SIZE = 12

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=SMALL_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('axes', titlesize=SMALL_SIZE)  # fontsize of the figure title
plt.rcParams['figure.dpi'] = 140

In [93]:
# Getting daily runoff results from simulations

results_dir = config.velma_data.parents[1] / 'results'

all_sims = natsorted(next(os.walk(results_dir))[1])

sims = []
runoff_sims = []
sim_results = []
for sim in all_sims:
    sim_dir = results_dir / sim
    files = os.listdir(sim_dir)
    if 'AnnualHydrologyResults.csv' not in files:
        continue
    sims.append(sim)
    velma_results = pd.read_csv(results_dir / sim / 'DailyResults.csv')
    runoff_sim = velma_results['Runoff_All(mm/day)_Delineated_Average']
    runoff_sims.append(runoff_sim)
        
# Format datetime of results
jday_pad = velma_results['Day'].apply(lambda x: str(x).zfill(3))
str_year = velma_results['Year'].apply(lambda x: str(x))
velma_results['date'] = str_year + jday_pad
rng = pd.to_datetime(velma_results['date'], format='%Y%j')

runoff_sims_stack = np.column_stack(runoff_sims)
runoff_sims_df = pd.DataFrame(data=runoff_sims_stack, columns=sims, index=rng)
runoff_sims_df['doy'], runoff_sims_df['year'] = runoff_sims_df.index.dayofyear, runoff_sims_df.index.year

# Get observed runoff
runoff_obs = pd.read_csv(config.velma_data / 'runoff' / 'ellsworth_Q_2004_2007.csv', names=['runoff_obs'])
runoff_obs.index = pd.date_range('01-01-2004', '12-31-2007')
runoff_obs['doy'], runoff_obs['year'] = runoff_obs.index.dayofyear, runoff_obs.index.year

In [94]:
# Convert runoff to pivot tables

runoff_sim_yearly = pd.pivot_table(runoff_sims_df, index=['doy'], columns=['year'],
                                   values=sims)

runoff_obs_yearly = pd.pivot_table(runoff_obs, index=['doy'], columns=['year'],
                                   values='runoff_obs')

## Plotting

In [96]:
# Plotting all simulations together

years = runoff_obs_yearly.columns.get_level_values(0)
fig, axes = plt.subplots(ncols=1, nrows=len(years), figsize=(6, 9))
for i, year in enumerate(years):
    runoff_obs_yearly.iloc[:, i].plot(ax=axes[i], label='Observed', linewidth=1, color='black')
    axes[i].set_title(year)
    axes[i].set_ylim([0, 80])
    for j, sim in enumerate(sims):
        col = (j * len(years)) + i
        runoff_sim_yearly.iloc[:, col].plot(ax=axes[i], label=sim, linewidth=0.4)
    leg = axes[0].legend(loc='upper left', bbox_to_anchor=(0, 2.5), fancybox=True, ncol=3)
    for line in leg.get_lines():
        line.set_linewidth(4.0)
axes[0].set_ylabel('Runoff (mm/day)')
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [99]:
# Interactively selection which sims to plot

# Plotting a subset of simulations

def plot_selected_sims(runoff_sim_yearly_selected, selected):    
    years = runoff_obs_yearly.columns.get_level_values(0)
    fig, axes = plt.subplots(ncols=1, nrows=len(years), figsize=(6, 9))
    for i, year in enumerate(years):
        runoff_obs_yearly.iloc[:, i].plot(ax=axes[i], label='Observed', linewidth=1, color='black')
        axes[i].set_title(year)
        axes[i].set_ylim([0, 80])
        for j, sim in enumerate(selected):
            col = (j * len(years)) + i
            runoff_sim_yearly_selected.iloc[:, col].plot(ax=axes[i], label=sim, linewidth=0.6)
    leg = axes[0].legend(loc='upper left', bbox_to_anchor=(0, 1.3), fancybox=True, ncol=3)
    for line in leg.get_lines():
        line.set_linewidth(4.0)
    axes[0].set_ylabel('Runoff (mm/day)')
    plt.tight_layout()


sel_mul = widgets.SelectMultiple(description="Simulations", options=sims, value=[sims[0]], disabled=False)
@interact(selected=sel_mul)
def interactive_plot(selected):
    tuples = pd.MultiIndex.from_product([selected, years])
    runoff_sim_yearly_sub = runoff_sim_yearly[tuples]
    plot_selected_sims(runoff_sim_yearly_sub, selected)



interactive(children=(SelectMultiple(description='Simulations', index=(0,), options=('ellsworth_baseline_04_07…

In [98]:
plt.close('all')

In [64]:
# Plotting a subset of simulations manually

sims_sub = ['ellsworth_baseline_04_07_9',
            'ellsworth_baseline_04_07_10',
            'ellsworth_baseline_04_07_11']

tuples = pd.MultiIndex.from_product([sims_sub, years])

runoff_sim_yearly_sub = runoff_sim_yearly[tuples]

years = runoff_obs_yearly.columns.get_level_values(0)
fig, axes = plt.subplots(ncols=1, nrows=len(years), figsize=(6, 9))
for i, year in enumerate(years):
    runoff_obs_yearly.iloc[:, i].plot(ax=axes[i], label='Observed', linewidth=1, color='black')
    axes[i].set_title(year)
    axes[i].set_ylim([0, 80])
    for j, sim in enumerate(sims_sub):
        col = (j * len(years)) + i
        runoff_sim_yearly_sub.iloc[:, col].plot(ax=axes[i], label=sim, linewidth=0.4)
axes[0].legend(loc='upper left', bbox_to_anchor=(0, 2.1), fancybox=True, ncol=2)
axes[0].set_ylabel('Runoff (mm/day)')
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …